In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import networkx as nx

# 데이터 로드
learner_data_path = r'D:\다운로드\thirdstu.csv'
relationship_data_path = r'D:\다운로드\label.csv'

learner_data = pd.read_csv(learner_data_path)
relationship_data = pd.read_csv(relationship_data_path)

# 'A'를 제거하고 고유한 숫자 ID로 변환
learner_data['assessmentItemID'] = learner_data['assessmentItemID'].apply(lambda x: int(x[1:]))

# assessmentItemID의 최대값을 확인
max_assessment_id = learner_data['assessmentItemID'].max()


In [2]:
# 그래프 생성 함수
def create_graph(relationship_data):
    graph = nx.DiGraph()
    for _, row in relationship_data.iterrows():
        from_node = row['from_id']
        to_node = row['to_id']
        graph.add_node(from_node)
        graph.add_node(to_node)
        graph.add_edge(from_node, to_node, weight=1.0)  # 가중치 기본값 1.0
    return graph

graph = create_graph(relationship_data)


In [3]:
# Dataset 클래스 정의
class LearnerDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.groups = data.groupby('learnerID')
        self.responses = []

        # 각 학습자의 데이터를 정리
        for _, group in self.groups:
            group = group.sort_values('assessmentItemID')  # 평가 문항 ID 순으로 정렬
            questions = group['assessmentItemID'].tolist()
            answers = group['answerCode'].tolist()

            # 정수형으로 변환
            questions = list(map(int, questions))
            answers = list(map(int, answers))

            self.responses.append(list(zip(questions, answers)))

    def __len__(self):
        return len(self.responses)

    def __getitem__(self, idx):
        questions, answers = zip(*self.responses[idx])
        questions = torch.tensor(questions, dtype=torch.long)
        answers = torch.tensor(answers, dtype=torch.float)
        return questions, answers

# 패딩 및 데이터 로더 정의
def collate_fn(batch):
    questions = [item[0] for item in batch]
    answers = [item[1] for item in batch]
    questions_padded = pad_sequence(questions, batch_first=True, padding_value=0)
    answers_padded = pad_sequence(answers, batch_first=True, padding_value=0)
    return questions_padded, answers_padded

# 학습자 데이터 로더 준비
batch_size = 8  # 배치 크기를 줄임
learner_dataset = LearnerDataset(learner_data)
learner_loader = DataLoader(learner_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)


In [4]:
# GKT 모델 정의
import torch.nn as nn
import torch.nn.functional as F

class GKT(nn.Module):
    def __init__(self, ku_num, graph, latent_dim=64, hidden_num=64, dropout=0.2, max_assessment_id=1000):
        super(GKT, self).__init__()
        self.ku_num = ku_num
        self.hidden_num = hidden_num
        self.latent_dim = latent_dim
        self.graph = graph

        # 임베딩 크기를 max_assessment_id + 1로 설정하여 인덱스 초과를 방지
        self.rnn = nn.GRU(input_size=latent_dim, hidden_size=hidden_num, batch_first=True)
        self.response_embedding = nn.Embedding(2 * (max_assessment_id + 1), latent_dim)
        self.concept_embedding = nn.Embedding(max_assessment_id + 1, latent_dim)
        self.f_self = nn.Linear(hidden_num + latent_dim, hidden_num)
        self.n_out = nn.Linear(hidden_num + latent_dim, hidden_num)
        self.n_in = nn.Linear(hidden_num + latent_dim, hidden_num)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_num, 1)

    def forward(self, questions, answers):
        batch_size, seq_len = questions.size()
        device = questions.device

        h0 = torch.zeros(1, batch_size, self.hidden_num).to(device)
        embedded_questions = self.concept_embedding(questions)
        embedded_answers = self.response_embedding(answers.long() + questions * 2)

        rnn_input = embedded_answers
        outputs, _ = self.rnn(rnn_input, h0)

        outputs = self.out(self.dropout(outputs)).squeeze(-1)
        outputs = torch.sigmoid(outputs)
        return outputs

In [5]:
# 모델 초기화 및 학습
ku_num = len(graph.nodes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
gkt_model = GKT(ku_num=ku_num, graph=graph, latent_dim=32, hidden_num=32, dropout=0.2, max_assessment_id=max_assessment_id).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(gkt_model.parameters(), lr=0.001)

# 모델 학습 함수
def train_gkt_model(model, data_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for questions, answers in data_loader:
            questions, answers = questions.to(device), answers.to(device)

            optimizer.zero_grad()
            outputs = model(questions, answers)
            loss = criterion(outputs, answers)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(data_loader)}')

# 모델 학습 실행
train_gkt_model(gkt_model, learner_loader)

cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.20 GiB. GPU 

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import networkx as nx
from torch.cuda.amp import autocast, GradScaler
import gc

# 데이터 로드
learner_data_path = r'D:\다운로드\thirdstu.csv'
relationship_data_path = r'D:\다운로드\label.csv'

learner_data = pd.read_csv(learner_data_path)
relationship_data = pd.read_csv(relationship_data_path)

# 'A'를 제거하고 고유한 숫자 ID로 변환
learner_data['assessmentItemID'] = learner_data['assessmentItemID'].apply(lambda x: int(x[1:]))

# assessmentItemID의 최대값을 확인
max_assessment_id = learner_data['assessmentItemID'].max()

# 그래프 생성 함수
def create_graph(relationship_data):
    graph = nx.DiGraph()
    for _, row in relationship_data.iterrows():
        from_node = row['from_id']
        to_node = row['to_id']
        graph.add_node(from_node)
        graph.add_node(to_node)
        graph.add_edge(from_node, to_node, weight=1.0)  # 가중치 기본값 1.0
    return graph

graph = create_graph(relationship_data)

# Dataset 클래스 정의
class LearnerDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.groups = data.groupby('learnerID')
        self.responses = []

        # 각 학습자의 데이터를 정리
        for _, group in self.groups:
            group = group.sort_values('assessmentItemID')  # 평가 문항 ID 순으로 정렬
            questions = group['assessmentItemID'].tolist()
            answers = group['answerCode'].tolist()

            # 정수형으로 변환
            questions = list(map(int, questions))
            answers = list(map(int, answers))

            self.responses.append(list(zip(questions, answers)))

    def __len__(self):
        return len(self.responses)

    def __getitem__(self, idx):
        questions, answers = zip(*self.responses[idx])
        questions = torch.tensor(questions, dtype=torch.long)
        answers = torch.tensor(answers, dtype=torch.float)
        return questions, answers

# 패딩 및 데이터 로더 정의
def collate_fn(batch):
    questions = [item[0] for item in batch]
    answers = [item[1] for item in batch]
    questions_padded = pad_sequence(questions, batch_first=True, padding_value=0)
    answers_padded = pad_sequence(answers, batch_first=True, padding_value=0)
    return questions_padded, answers_padded

# 학습자 데이터 로더 준비
batch_size = 2  # 배치 크기를 가능한 최소값으로 줄임
learner_dataset = LearnerDataset(learner_data)
learner_loader = DataLoader(learner_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

# GKT 모델 정의
import torch.nn as nn

class GKT(nn.Module):
    def __init__(self, ku_num, graph, latent_dim=16, hidden_num=16, dropout=0.2, max_assessment_id=1000):
        super(GKT, self).__init__()
        self.ku_num = ku_num
        self.hidden_num = hidden_num
        self.latent_dim = latent_dim
        self.graph = graph

        # 임베딩 크기를 max_assessment_id + 1로 설정하여 인덱스 초과를 방지
        self.rnn = nn.GRU(input_size=latent_dim, hidden_size=hidden_num, batch_first=True)
        self.response_embedding = nn.Embedding(2 * (max_assessment_id + 1), latent_dim)
        self.concept_embedding = nn.Embedding(max_assessment_id + 1, latent_dim)
        self.f_self = nn.Linear(hidden_num + latent_dim, hidden_num)
        self.n_out = nn.Linear(hidden_num + latent_dim, hidden_num)
        self.n_in = nn.Linear(hidden_num + latent_dim, hidden_num)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_num, 1)

    def forward(self, questions, answers):
        batch_size, seq_len = questions.size()
        device = questions.device

        h0 = torch.zeros(1, batch_size, self.hidden_num).to(device)
        embedded_questions = self.concept_embedding(questions)
        embedded_answers = self.response_embedding(answers.long() + questions * 2)

        rnn_input = embedded_answers
        outputs, _ = self.rnn(rnn_input, h0)

        outputs = self.out(self.dropout(outputs)).squeeze(-1)
        # BCEWithLogitsLoss를 사용하므로 시그모이드 적용하지 않음
        return outputs

# 모델 초기화 및 학습
ku_num = len(graph.nodes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gkt_model = GKT(ku_num=ku_num, graph=graph, latent_dim=16, hidden_num=16, dropout=0.2, max_assessment_id=max_assessment_id).to(device)

# 손실 함수를 BCEWithLogitsLoss로 변경
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gkt_model.parameters(), lr=0.001)

# Mixed Precision Training을 위한 GradScaler 초기화
scaler = GradScaler()

# 모델 학습 함수
def train_gkt_model(model, data_loader, epochs=10, accumulation_steps=4):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        optimizer.zero_grad()
        for i, (questions, answers) in enumerate(data_loader):
            questions, answers = questions.to(device), answers.to(device)

            # Mixed Precision Training 사용
            with autocast():
                outputs = model(questions, answers)
                loss = criterion(outputs, answers) / accumulation_steps

            # 스케일링을 통한 역전파 및 최적화
            scaler.scale(loss).backward()

            # Gradient Accumulation 적용
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(data_loader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            epoch_loss += loss.item()

            # 메모리 관리 및 캐시 정리
            torch.cuda.empty_cache()
            gc.collect()
        
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(data_loader)}')

# 모델 학습 실행
train_gkt_model(gkt_model, learner_loader)


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.60 GiB. GPU 

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 데이터 불러오기
label_data = pd.read_csv(r'D:\다운로드\label.csv')
thirdstu_data = pd.read_csv(r'D:\다운로드\thirdstu.csv')

# 데이터 전처리
edges = label_data[['from_id', 'to_id']].drop_duplicates()
nodes = pd.concat([edges['from_id'], edges['to_id']]).unique()
learner_data = thirdstu_data[['learnerID', 'assessmentItemID', 'answerCode', 'knowledgeTag']]
merged_data = learner_data.merge(label_data, left_on='knowledgeTag', right_on='from_id', how='left')
print(merged_data.head())

# 훈련 및 테스트 데이터 분리
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# 그래프 생성
graph = nx.DiGraph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges.values)

# 임베딩 학습
stellar_graph = StellarGraph.from_networkx(graph)
rw = BiasedRandomWalk(stellar_graph)
walks = rw.run(nodes=list(graph.nodes()), length=100, n=10, p=0.5, q=2.0)
str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=128, window=5, min_count=1, sg=1, workers=4)
node_embeddings = {node: model.wv[str(node)] for node in graph.nodes()}

# 임베딩 데이터를 활용한 입력 생성 함수
def create_input_data(data, node_embeddings):
    X = np.array([node_embeddings.get(str(item), np.zeros(128)) for item in data['assessmentItemID']])
    y = data['answerCode'].values
    return X, y

# 훈련 및 테스트 데이터 준비
X_train, y_train = create_input_data(train_data, node_embeddings)
X_test, y_test = create_input_data(test_data, node_embeddings)

# LSTM에 입력할 데이터를 3차원으로 변환
X_train = np.expand_dims(X_train, axis=1)  # (batch_size, time_steps, features)
X_test = np.expand_dims(X_test, axis=1)

# PyTorch 데이터셋 및 데이터로더 생성
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# PyTorch 모델 정의
class KSGKTModel(nn.Module):
    def __init__(self):
        super(KSGKTModel, self).__init__()
        self.lstm = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.attention = nn.MultiheadAttention(embed_dim=256, num_heads=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out.permute(1, 0, 2)  # LSTM output shape adjustment for attention
        attn_output, _ = self.attention(lstm_out, lstm_out, lstm_out)
        attn_output = attn_output.permute(1, 0, 2)  # Shape back to (batch, seq, feature)
        flatten_out = self.flatten(attn_output)
        x = torch.relu(self.fc1(flatten_out))
        x = self.sigmoid(self.fc2(x))
        return x

# 모델 초기화
model = KSGKTModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습 함수
def train(model, train_loader, criterion, optimizer):
    model.train()
    for data, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

# 모델 평가 함수
def evaluate(model, test_loader, criterion):
    model.eval()
    loss_total = 0
    correct = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            loss = criterion(outputs.squeeze(), labels)
            loss_total += loss.item()
            preds = (outputs.squeeze() > 0.5).float()
            correct += (preds == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return loss_total / len(test_loader), accuracy

# 학습 및 평가 루프
for epoch in range(10):
    train(model, train_loader, criterion, optimizer)
    loss, accuracy = evaluate(model, test_loader, criterion)
    print(f'Epoch {epoch+1}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}')


    learnerID assessmentItemID  answerCode  knowledgeTag  from_id  \
0  A030000005       A030001001           1           307    307.0   
1  A030000005       A030001001           1           307    307.0   
2  A030000005       A030001001           1           307    307.0   
3  A030000005       A030001002           1           307    307.0   
4  A030000005       A030001002           1           307    307.0   

                     from_name from_semester  \
0  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
1  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
2  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
3  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
4  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   

                                    from_description  from_chapter_id  \
0  1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...            149.0   
1  1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...            149.0   
2  1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...            149.0   
3  1

In [4]:
merged_data

,learnerID,assessmentItemID,answerCode,knowledgeTag,from_id,from_name,from_semester,from_description,from_chapter_id,from_chapter_name,from_achievement_id,from_achievement_name,to_id,to_name,to_semester,to_description,to_chapter_id,to_chapter_name,to_achievement_id,to_achievement_name
0,A030000005,A030001001,1,307,307.0,받아 내림이 없는 $(세 자릿수)-(세 자릿수)$,초등-초3-1학기,"1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...",149.0,덧셈과 뺄셈 > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...,7797.0,(몇십 몇)-(몇),초등-초1-2학기,(1) 낱개끼리 줄을 맞추어 씁니다.\n(2) 낱개는 낱개끼리 빼고 10개씩 묶음...,42.0,덧셈과 뺄셈(1) > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...
1,A030000005,A030001001,1,307,307.0,받아 내림이 없는 $(세 자릿수)-(세 자릿수)$,초등-초3-1학기,"1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...",149.0,덧셈과 뺄셈 > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...,7798.0,(몇십)-(몇십),초등-초1-2학기,"(1) 10개씩 묶음은 10개씩 묶음끼리, 낱개는 낱개끼리 줄을 맞추어 씁니다.\n...",43.0,덧셈과 뺄셈(1) > 뺄셈을 해 볼까요 (2),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...
2,A030000005,A030001001,1,307,307.0,받아 내림이 없는 $(세 자릿수)-(세 자릿수)$,초등-초3-1학기,"1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...",149.0,덧셈과 뺄셈 > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...,7799.0,(몇십 몇)-(몇십 몇),초등-초1-2학기,"(1) 10개씩 묶음은 10개씩 묶음끼리, 낱개는 낱개끼리 줄을 맞추어 씁니다.\n...",44.0,덧셈과 뺄셈(1) > 뺄셈을 해 볼까요 (3),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...
3,A030000005,A030001002,1,307,307.0,받아 내림이 없는 $(세 자릿수)-(세 자릿수)$,초등-초3-1학기,"1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...",149.0,덧셈과 뺄셈 > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...,7797.0,(몇십 몇)-(몇),초등-초1-2학기,(1) 낱개끼리 줄을 맞추어 씁니다.\n(2) 낱개는 낱개끼리 빼고 10개씩 묶음...,42.0,덧셈과 뺄셈(1) > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...
4,A030000005,A030001002,1,307,307.0,받아 내림이 없는 $(세 자릿수)-(세 자릿수)$,초등-초3-1학기,"1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...",149.0,덧셈과 뺄셈 > 뺄셈을 해 볼까요 (1),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...,7798.0,(몇십)-(몇십),초등-초1-2학기,"(1) 10개씩 묶음은 10개씩 묶음끼리, 낱개는 낱개끼리 줄을 맞추어 씁니다.\n...",43.0,덧셈과 뺄셈(1) > 뺄셈을 해 볼까요 (2),45.0,두 자릿수의 범위에서 받아 올림이 없는 덧셈과 받아 내림이 없는 뺄셈의 계산 원리를...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575612,A030003000,A030198004,0,1984,1984.0,그림그래프,초등-초3-2학기,알려고 하는 수(조사한 수)를 그림으로 나타낸 그래프,219.0,자료의 정리 > 그림그래프를 알아볼까요,295.0,실생활 자료를 수집하여 그림그래프로 나타낼 수 있다.,8135.0,자료를 보고 표로 나타내기,초등-초2-2학기,자료를 분류하여 분류한 종류에 맞게 수를 세어 표로 나타냅니다.\n자료를 보고 바로...,135.0,표와 그래프 > 자료를 보고 표로 나타내어 볼까요,293.0,"분류한 자료를 표로 나타내고, 표로 나타내면 편리한 점을 말할 수 있다."
575613,A030003000,A030198004,0,1984,1984.0,그림그래프,초등-초3-2학기,알려고 하는 수(조사한 수)를 그림으로 나타낸 그래프,219.0,자료의 정리 > 그림그래프를 알아볼까요,295.0,실생활 자료를 수집하여 그림그래프로 나타낼 수 있다.,8138.0,표와 그래프의 편리한 점,초등-초2-2학기,표는 종류별 자료의 수와 전체 자료의 수를 쉽게 알 수 있습니다.\n그래프는 가장 ...,138.0,표와 그래프 > 표와 그래프의 내용을 알아볼까요,NaN,NaN
575614,A030003000,A030198005,1,1984,1984.0,그림그래프,초등-초3-2학기,알려고 하는 수(조사한 수)를 그림으로 나타낸 그래프,219.0,자료의 정리 > 그림그래프를 알아볼까요,295.0,실생활 자료를 수집하여 그림그래프로 나타낼 수 있다.,1983.0,자료를 수집하여 표로 나타내는 방법,초등-초3-2학기,1. 조사할 내용 정하기\n2. 자료를 수집할 방법 정하기\n3. 자료 수집하기\n...,218.0,자료의 정리 > 자료를 수집하여 표로 나타내어 볼까요,293.0,"분류한 자료를 표로 나타내고, 표로 나타내면 편리한 점을 말할 수 있다."
575615,A030003000,A030198005,1,1984,1984.0,그림그래프,초등-초3-2학기,알려고 하는 수(조사한 수)를 그림으로 나타낸 그래프,219.0,자료의 정리 > 그림그래프를 알아볼까요,295.0,실생활 자료를 수집하여 그림그래프로 나타낼 수 있다.,8135.0,자료를 보고 표로 나타내기,초등-초2-2학기,자료를 분류하여 분류한 종류에 맞게 수를 세어 표로 나타냅니다.\n자료를 보고 바로...,135.0,표와 그래프 > 자료를 보고 표로 나타내어 볼까요,293.0,"분류한 자료를 표로 나타내고, 표로 나타내면 편리한 점을 말할 수 있다."


In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 데이터 불러오기
label_data = pd.read_csv(r'D:\다운로드\label.csv')
thirdstu_data = pd.read_csv(r'D:\다운로드\thirdstu.csv')

# 데이터 전처리
edges = label_data[['from_id', 'to_id']].drop_duplicates()
nodes = pd.concat([edges['from_id'], edges['to_id']]).unique()
learner_data = thirdstu_data[['learnerID', 'assessmentItemID', 'answerCode', 'knowledgeTag']]
merged_data = learner_data.merge(label_data, left_on='knowledgeTag', right_on='from_id', how='left')
print(merged_data.head())

# 훈련 및 테스트 데이터 분리
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# 그래프 생성
graph = nx.DiGraph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges.values)

# 임베딩 학습
stellar_graph = StellarGraph.from_networkx(graph)
rw = BiasedRandomWalk(stellar_graph)
walks = rw.run(nodes=list(graph.nodes()), length=100, n=20, p=0.7, q=2.5)
str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=128, window=7, min_count=1, sg=1, workers=-1)
node_embeddings = {node: model.wv[str(node)] for node in graph.nodes()}

# 임베딩 데이터를 활용한 입력 생성 함수
def create_input_data(data, node_embeddings):
    X = np.array([node_embeddings.get(str(item), np.zeros(128)) for item in data['assessmentItemID']])
    y = data['answerCode'].values
    return X, y

# 훈련 및 테스트 데이터 준비
X_train, y_train = create_input_data(train_data, node_embeddings)
X_test, y_test = create_input_data(test_data, node_embeddings)

# LSTM에 입력할 데이터를 3차원으로 변환
X_train = np.expand_dims(X_train, axis=1)  # (batch_size, time_steps, features)
X_test = np.expand_dims(X_test, axis=1)

# PyTorch 데이터셋 및 데이터로더 생성
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# PyTorch 모델 정의
class KSGKTModel(nn.Module):
    def __init__(self):
        super(KSGKTModel, self).__init__()
        self.lstm = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.attention = nn.MultiheadAttention(embed_dim=256, num_heads=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256, 64)
        self.dropout = nn.Dropout(0.3)  # Dropout 추가
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out.permute(1, 0, 2)  # LSTM output shape adjustment for attention
        attn_output, _ = self.attention(lstm_out, lstm_out, lstm_out)
        attn_output = attn_output.permute(1, 0, 2)  # Shape back to (batch, seq, feature)
        flatten_out = self.flatten(attn_output)
        x = torch.relu(self.fc1(flatten_out))
        x = self.dropout(x)  # Dropout 사용
        x = self.sigmoid(self.fc2(x))
        return x

# 모델 초기화
model = KSGKTModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 학습률 변경 가능

# 학습 및 평가 루프
def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        for data, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

def evaluate(model, test_loader, criterion):
    model.eval()
    loss_total = 0
    correct = 0
    with torch.no_grad():
        for data, labels in test_loader:
            outputs = model(data)
            loss = criterion(outputs.squeeze(), labels)
            loss_total += loss.item()
            preds = (outputs.squeeze() > 0.5).float()
            correct += (preds == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return loss_total / len(test_loader), accuracy

# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 학습 및 평가
train(model, train_loader, criterion, optimizer, num_epochs=20)
loss, accuracy = evaluate(model, test_loader, criterion)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

c:\ProgramData\anaconda3\envs\GKT\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


    learnerID assessmentItemID  answerCode  knowledgeTag  from_id  \
0  A030000005       A030001001           1           307    307.0   
1  A030000005       A030001001           1           307    307.0   
2  A030000005       A030001001           1           307    307.0   
3  A030000005       A030001002           1           307    307.0   
4  A030000005       A030001002           1           307    307.0   

                     from_name from_semester  \
0  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
1  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
2  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
3  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   
4  받아 내림이 없는 $(세 자릿수)-(세 자릿수)$     초등-초3-1학기   

                                    from_description  from_chapter_id  \
0  1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...            149.0   
1  1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...            149.0   
2  1. 각 자리의 숫자를 맞추어 적습니다.\n2. 일의 자리, 십의 자리, 백의 자리...            149.0   
3  1

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec

# 데이터 불러오기
label_data = pd.read_csv(r'D:\다운로드\label.csv')
thirdstu_data = pd.read_csv(r'D:\다운로드\thirdstu.csv')

# 그래프 생성 및 임베딩 학습
edges = label_data[['from_id', 'to_id']].drop_duplicates()
graph = nx.DiGraph()
graph.add_edges_from(edges.values)
stellar_graph = StellarGraph.from_networkx(graph)

rw = BiasedRandomWalk(stellar_graph)
walks = rw.run(nodes=list(graph.nodes()), length=100, n=20, p=0.7, q=2.5)
str_walks = [[str(n) for n in walk] for walk in walks]
word2vec_model = Word2Vec(str_walks, vector_size=128, window=7, min_count=1, sg=1, workers=-1)
node_embeddings = {node: word2vec_model.wv[str(node)] for node in graph.nodes()}

# 학습자 평가 데이터 준비
learner_data = thirdstu_data[['learnerID', 'assessmentItemID', 'answerCode', 'knowledgeTag']]
merged_data = learner_data.merge(label_data, left_on='knowledgeTag', right_on='from_id', how='left')

# 훈련 및 테스트 데이터 분리
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)

# 데이터 준비 함수
def create_input_data(data, node_embeddings):
    X = np.array([node_embeddings.get(str(item), np.zeros(128)) for item in data['assessmentItemID']])
    y = data['answerCode'].values
    return X, y

# 데이터 로드 및 전처리
X_train, y_train = create_input_data(train_data, node_embeddings)
X_test, y_test = create_input_data(test_data, node_embeddings)

# PyTorch 텐서로 변환
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # (batch_size, time_steps=1, features=128)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# DataLoader 정의
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# KSGKT 모델 정의
class KSGKTModel(nn.Module):
    def __init__(self):
        super(KSGKTModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=128, hidden_size=256, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=256, hidden_size=128, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)
        self.attention = nn.MultiheadAttention(embed_dim=64, num_heads=1, batch_first=True)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.attention(x, x, x)  # Q, K, V are all the LSTM output
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.output(x))
        return x

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = KSGKTModel().to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
def train(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

# 모델 평가
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            all_predictions.extend(predicted.cpu().numpy())
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return running_loss / len(loader), accuracy, np.array(all_predictions)

# 학습 루프
epochs = 50
for epoch in range(epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy, predictions = evaluate(model, test_loader, criterion, device)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# 학습자의 지식 상태 예측
test_data['predicted_proficiency'] = predictions

# 특정 개념에 대해 학습자가 자주 틀리거나 낮은 예측 확률을 보이는 경우 식별
def identify_weak_concepts(data, threshold=0.5):
    weak_concepts = data[data['predicted_proficiency'] < threshold]['knowledgeTag'].unique()
    return weak_concepts

# 높은 이해도를 보이는 개념에 대한 후수학습 제시
def suggest_followup_learning(concept_id, prerequisite_data):
    return prerequisite_data[prerequisite_data['from_id'] == concept_id]['to_id'].values

# 낮은 이해도를 보이는 개념에 대한 선수학습 제시
def suggest_prerequisite_learning(concept_id, prerequisite_data):
    return prerequisite_data[prerequisite_data['to_id'] == concept_id]['from_id'].values

# 복습 제안: 반복적인 실수를 하는 개념 식별
def suggest_review(data, error_threshold=0.3):
    review_concepts = data[data['predicted_proficiency'] < error_threshold]['knowledgeTag'].unique()
    return review_concepts

# 학습자에게 제안할 선수 및 후수학습 결정
weak_concepts = identify_weak_concepts(test_data)
strong_concepts = test_data[test_data['predicted_proficiency'] >= 0.8]['knowledgeTag'].unique()
review_concepts = suggest_review(test_data)

# 제안된 학습 경로 출력
for concept in weak_concepts:
    prerequisites = suggest_prerequisite_learning(concept, label_data)
    print(f"개념 '{concept}'에 대한 부족한 이해를 보완하기 위해 제안된 선수학습: {prerequisites}")

for concept in strong_concepts:
    followups = suggest_followup_learning(concept, label_data)
    print(f"개념 '{concept}'에 대한 충분한 이해를 기반으로 제안된 후수학습: {followups}")

for concept in review_concepts:
    print(f"개념 '{concept}'에 대한 반복적인 실수가 식별되어 복습을 제안합니다.")


Epoch 1, Train Loss: 0.5989, Test Loss: 0.5976, Test Accuracy: 0.7155
Epoch 2, Train Loss: 0.5983, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 3, Train Loss: 0.5982, Test Loss: 0.5978, Test Accuracy: 0.7155
Epoch 4, Train Loss: 0.5982, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 5, Train Loss: 0.5982, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 6, Train Loss: 0.5981, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 7, Train Loss: 0.5982, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 8, Train Loss: 0.5981, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 9, Train Loss: 0.5981, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 10, Train Loss: 0.5981, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 11, Train Loss: 0.5981, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 12, Train Loss: 0.5981, Test Loss: 0.5973, Test Accuracy: 0.7155
Epoch 13, Train Loss: 0.5981, Test Loss: 0.5972, Test Accuracy: 0.7155
Epoch 14, Train Loss: 0.5981, Test Loss: 0.5973, Test Accuracy: 0.7155
Epoch 15, Train

C:\Users\USER\AppData\Local\Temp\ipykernel_17556\3468450319.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_proficiency'] = predictions


In [9]:
# 특정 학습자에게 제안할 학습 경로 제공 함수
def suggest_learning_path_for_learner(learner_id, test_data, label_data):
    # 학습자의 데이터 필터링
    learner_data = test_data[test_data['learnerID'] == learner_id].copy()

    # 학습자의 지식 상태 예측 결과 추가
    learner_data.loc[:, 'predicted_proficiency'] = predictions[test_data['learnerID'] == learner_id]

    # 특정 개념에 대해 학습자가 자주 틀리거나 낮은 예측 확률을 보이는 경우 식별
    weak_concepts = identify_weak_concepts(learner_data, threshold=0.2)
    strong_concepts = learner_data[learner_data['predicted_proficiency'] >= 0.8]['knowledgeTag'].unique()
    review_concepts = suggest_review(learner_data, error_threshold=0.3)

    # 제안된 학습 경로 출력
    print(f"학습자 '{learner_id}'의 학습 경로 제안:")
    
    if weak_concepts.size > 0:
        for concept in weak_concepts:
            prerequisites = suggest_prerequisite_learning(concept, label_data)
            print(f"개념 '{concept}'에 대한 부족한 이해를 보완하기 위해 제안된 선수학습: {prerequisites}")
    else:
        print("선수학습을 제안할 개념이 없습니다.")
    
    if strong_concepts.size > 0:
        for concept in strong_concepts:
            followups = suggest_followup_learning(concept, label_data)
            print(f"개념 '{concept}'에 대한 충분한 이해를 기반으로 제안된 후수학습: {followups}")
    else:
        print("후수학습을 제안할 개념이 없습니다.")
    
    if review_concepts.size > 0:
        for concept in review_concepts:
            print(f"개념 '{concept}'에 대한 반복적인 실수가 식별되어 복습을 제안합니다.")
    else:
        print("복습을 제안할 개념이 없습니다.")

# 특정 학습자의 학습 경로 제안 호출
learner_id = 'A030000385'  # 예시 learnerID
suggest_learning_path_for_learner(learner_id, test_data, label_data)


NameError: name 'threshold' is not defined

In [11]:
# 특정 개념에 대해 학습자가 자주 틀리거나 낮은 예측 확률을 보이는 경우 식별
def identify_weak_concepts(data, threshold=0.3):
    weak_concepts = data[data['predicted_proficiency'] <= threshold]['knowledgeTag'].unique()
    return weak_concepts

# 학습자에게 제안할 선수 및 후수학습 결정
weak_concepts = identify_weak_concepts(test_data, threshold=0.3)  # 여기에서 threshold를 0.3 이하로 변경
strong_concepts = test_data[test_data['predicted_proficiency'] >= 0.9]['knowledgeTag'].unique()
review_concepts = suggest_review(test_data)

# 제안된 학습 경로 출력
for concept in weak_concepts:
    prerequisites = suggest_prerequisite_learning(concept, label_data)
    print(f"개념 '{concept}'에 대한 부족한 이해를 보완하기 위해 제안된 선수학습: {prerequisites}")

for concept in strong_concepts:
    followups = suggest_followup_learning(concept, label_data)
    print(f"개념 '{concept}'에 대한 충분한 이해를 기반으로 제안된 후수학습: {followups}")

for concept in review_concepts:
    print(f"개념 '{concept}'에 대한 반복적인 실수가 식별되어 복습을 제안합니다.")


개념 '407'에 대한 충분한 이해를 기반으로 제안된 후수학습: [ 405 8006 8014 8015 8017 8019 8020 8021 8016 8018 8022  373]
개념 '7653'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8122 7636]
개념 '373'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8006 8014 8015 8017 8019 8020 8021 8016 8018]
개념 '452'에 대한 충분한 이해를 기반으로 제안된 후수학습: [ 351  447 8122]
개념 '409'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8006 8014 8015 8017 8019 8020 8021 8016 8018 8022  373  405]
개념 '7636'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8006 8014 8015 8017 8019 8020 8021 8016 8018 8022  405]
개념 '1982'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8138 8091]
개념 '342'에 대한 충분한 이해를 기반으로 제안된 후수학습: [6804  332]
개념 '371'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8006 8014 8015 8017 8019 8020 8021 8016 8018 8022]
개념 '339'에 대한 충분한 이해를 기반으로 제안된 후수학습: [6805  332]
개념 '334'에 대한 충분한 이해를 기반으로 제안된 후수학습: []
개념 '465'에 대한 충분한 이해를 기반으로 제안된 후수학습: [5836 6648]
개념 '405'에 대한 충분한 이해를 기반으로 제안된 후수학습: [8006 8014 8015 8017 8019 8020 8021 8016 8018 8022]
개념 '346'에 대한 충분한 이해를 기반으로 제안된 후수학습: [6804  332  342]
개념 '332'에 대한 충분한 이해를 기반으로 제안된 후수학습: []
개념 '7781'에 대한 충분한 이해를 기반으